In [1]:
import pandas as pd
import numpy as np
# for pretty printing dfs
from IPython.display import display

In [52]:
trials = pd.read_json('./dynomite_trials.json')
mods = pd.read_json('./dynomite_mods.json')
# replace numbers with human_readable 
index_to_modname = {
        '1':'boost', '2':'fuel',
        '3':'displacement', '4':'turbo_class',
        '5':'turbo_name', '7':'injection', '8':'nitrous',
        '9':'correction', '10':'heads',
        '11':'max_hp', '12':'max_tq'
};
readable_mods = [{index_to_modname[k]:v for k, v in r['mods'].items()} for idx, r in trials.iterrows()]
trials['mods'] = readable_mods
# split into hp/torque trials - make copies because we will add cols later
hp_trials = trials.copy().loc[trials['data_type_id'] == 1]
torque_trials = trials.copy().loc[trials['data_type_id'] == 2]
trials.head()

,display_info,name,displayName,owner_id,car_id,run_id,data_type_id,mods,run_data
0,{'username': '11secondFWD'},11secondFWD HP 9 1,11secondFWD HP (17g),9,9,1,1,"{'turbo_class': 'td04', 'turbo_name': '17g'}","[{'x': 2510, 'y': 53}, {'x': 2921, 'y': 107}, ..."
1,{'username': '11secondFWD'},11secondFWD TQ 9 1,11secondFWD TQ (17g),9,9,1,2,"{'turbo_class': 'td04', 'turbo_name': '17g'}","[{'x': 2514, 'y': 118}, {'x': 2619, 'y': 143},..."
2,{'username': '2root4u'},2root4u HP 18 1,2root4u HP (gt368),18,18,1,1,"{'boost': '25', 'turbo_class': 'td04', 'turbo_...","[{'x': 3564, 'y': 210}, {'x': 3942, 'y': 262},..."
3,{'username': '3sx'},3sx HP 6 1,3sx HP (GT35),6,6,1,1,"{'turbo_class': 'td05', 'turbo_name': 'GT35', ...","[{'x': 4200, 'y': 200}, {'x': 4800, 'y': 370},..."
4,{'username': '97vr4'},97vr4 HP 15 1,97vr4 HP (19t),15,15,1,1,"{'boost': '25', 'turbo_class': 'td04', 'turbo_...","[{'x': 3457, 'y': 201}, {'x': 3686, 'y': 244},..."


In [3]:
# regular 5-figure stats summary
def q_summary(numbers):
    # to calculate outliers
    quartiles = [np.quantile(numbers, 1/4), np.quantile(numbers, 2/4), np.quantile(numbers, 3/4)]
    IQR = 1.5*(quartiles[2]-quartiles[0])
    outliers = [n for n in numbers if (n < quartiles[0]-IQR or n > quartiles[2]+IQR)]
    withoutOutliers = [n for n in numbers if n not in outliers]
    # now print
    ret = {
        'min':np.min(withoutOutliers),
        'q1':quartiles[0],
        'q2':quartiles[1],
        'q3':quartiles[2],
        'max':np.max(withoutOutliers),
        'outliers':outliers if outliers else None
    }
    return ret

# 5-figure stats summary that can handle dataframe series as input and returns df rows
def q_summary_dataframe(df, col):
    numbers = df[col]
    # quartiles calculated as numbers
    quartiles = [np.quantile(numbers, 1/4), np.quantile(numbers, 2/4), np.quantile(numbers, 3/4)]
    IQR = 1.5*(quartiles[2]-quartiles[0])
    # outliers are rows, not single numbers now. you need bitwise OR for loc, it's weird
    outliers = df.loc[(df[col] < quartiles[0]-IQR) | (df[col] > quartiles[2]+IQR)]
    # bitwise NOT ~, i.e. rows not in outliers
    without_outliers = df[~df.isin(outliers)]
    # now print
    ret = {
        'min':without_outliers[without_outliers[col] == without_outliers[col].min()],
        # get rows that match quartiles
        'q1':without_outliers[without_outliers[col] == quartiles[0]],
        'q2':without_outliers[without_outliers[col] == quartiles[1]],
        'q3':without_outliers[without_outliers[col] == quartiles[2]],
        'max':without_outliers[without_outliers[col] == without_outliers[col].max()],
        'outliers': outliers if not outliers.empty else None
    }
    return ret

# pretty print 5-figure df summary
def print_df_summary(summary):
    print('Min row')
    display(summary['min'])
    print('q1 row')
    display(summary['q1'])
    print('q2 row')
    display(summary['q2'])
    print('q3 row')
    display(summary['q3'])
    print('Max row')
    display(summary['max'])
    if summary['outliers'] is not None:
        print('Outlier rows')
        display(summary['outliers'])

# print out a 5-figure stats summary and return the integration bounds (median start and end rpm)
def summarize_trials_bounds(trials, name):
    start_rpms = [run[0]['x'] for run in trials] # only interested in 'x' part, i.e. rpm
    end_rpms = [run[-1]['x'] for run in trials]
    start_summary = q_summary(start_rpms)
    end_summary = q_summary(end_rpms)
    print('{} start RPM summary: {}'.format(name, start_summary))
    print('{} end RPM summary: {}'.format(name, end_summary))
    start = start_summary['q2']
    end = end_summary['q2']
    return (start, end)

# integrate run data [(x,y)...], filtering out values outside the [a,b] interval
def definite_discrete_integral(run, a, b):
    # filter out any data points with rpm less than a, greater than b
    filtered = [p for p in run if p['x'] >= a and p['x'] <= b]
    # discrete integral using trapezoidal rule
    xs = [p['x'] for p in filtered]
    ys = [p['y'] for p in filtered]
    return np.trapz(ys, x=xs)

## Statistical summary on runs (integral as aggregate function)
Run data has a different number of points for each run. 'outliers' is not a good question to ask without refinement because these are continous functions. We will do an outlier/min/max summary on the integral of each run, because a run with higher horsepower would have a bigger integral, which is what we are interested in.

Runs have different start and end points (RPM), so we will take a median of start/end to use as the integration bounds.

In [4]:
integral_col_name = "total_power_normalized"

# for horsepower
hp_start, hp_end = summarize_trials_bounds(hp_trials['run_data'], 'horsepower')
print('Integration bounds (median q2): ({}, {})'.format(hp_start, hp_end), end='\n\n')
# now do the integrals and add to dataframe
hp_trials[integral_col_name] = hp_trials['run_data'].apply(lambda r: definite_discrete_integral(r, hp_start, hp_end))
hp_trials.head()

horsepower start RPM summary: {'min': 1020, 'q1': 2510.0, 'q2': 3065.0, 'q3': 3603.0, 'max': 4672, 'outliers': None}
horsepower end RPM summary: {'min': 5988, 'q1': 6639.0, 'q2': 6965.0, 'q3': 7158.0, 'max': 7544, 'outliers': [8500, 7950, 8011, 8463, 8468, 8072, 8015]}
Integration bounds (median q2): (3065.0, 6965.0)



,display_info,name,displayName,owner_id,car_id,run_id,data_type_id,mods,run_data,total_power_normalized
0,{'username': '11secondFWD'},11secondFWD HP 9 1,11secondFWD HP (17g),9,9,1,1,"{'4': 'td04', '5': '17g'}","[{'x': 2510, 'y': 53}, {'x': 2921, 'y': 107}, ...",1023050.0
2,{'username': '2root4u'},2root4u HP 18 1,2root4u HP (gt368),18,18,1,1,"{'1': '25', '4': 'td04', '5': 'gt368'}","[{'x': 3564, 'y': 210}, {'x': 3942, 'y': 262},...",1354755.0
3,{'username': '3sx'},3sx HP 6 1,3sx HP (GT35),6,6,1,1,"{'4': 'td05', '5': 'GT35', '8': '100'}","[{'x': 4200, 'y': 200}, {'x': 4800, 'y': 370},...",1579750.0
4,{'username': '97vr4'},97vr4 HP 15 1,97vr4 HP (19t),15,15,1,1,"{'1': '25', '4': 'td04', '5': '19t'}","[{'x': 3457, 'y': 201}, {'x': 3686, 'y': 244},...",1820158.0
5,{'username': 'AdamVR4'},AdamVR4 HP 8 1,AdamVR4 HP (evo3 16g),8,8,1,1,"{'1': '30', '2': 'race', '4': 'td05', '5': 'ev...","[{'x': 3166, 'y': 144}, {'x': 3516, 'y': 189},...",1308968.5


In [5]:
# 5-stat summary on the hp integrals
hp_integral_summary = q_summary_dataframe(hp_trials, integral_col_name)
print_df_summary(hp_integral_summary)

Min row


,display_info,name,displayName,owner_id,car_id,run_id,data_type_id,mods,run_data,total_power_normalized
49,{'username': 'levani041491'},levani041491 HP 51 1,levani041491 HP (13t),47.0,51.0,1.0,1.0,"{'1': '8', '4': 'td04', '5': '13t', '11': '297...","[{'x': 2483, 'y': 103}, {'x': 3030, 'y': 155},...",747638.0


q1 row


,display_info,name,displayName,owner_id,car_id,run_id,data_type_id,mods,run_data,total_power_normalized
57,{'username': 'niteryder'},niteryder HP 35 1,niteryder HP (twim 50trim bb),33.0,35.0,1.0,1.0,"{'2': 'race', '4': 'td05', '5': 'twim 50trim bb'}","[{'x': 4503, 'y': 205}, {'x': 4716, 'y': 235},...",1164535.5


q2 row


...

,display_info,name,displayName,owner_id,car_id,run_id,data_type_id,mods,run_data,total_power_normalized
22,{'username': 'HillbillyHomeboy'},HillbillyHomeboy HP 37 1,HillbillyHomeboy HP (GT3076),34.0,37.0,1.0,1.0,"{'1': '31', '4': 'td05', '5': 'GT3076', '8': '...","[{'x': 3923, 'y': 211}, {'x': 4049, 'y': 242},...",1669246.0


Max row


,display_info,name,displayName,owner_id,car_id,run_id,data_type_id,mods,run_data,total_power_normalized
97,{'username': 'Trevor'},Trevor HP 42 2,Trevor HP (GTX4202R),39.0,42.0,2.0,1.0,"{'1': '33.1', '2': 'e85', '3': '3.7', '4': 't4...","[{'x': 2536, 'y': 106}, {'x': 2828, 'y': 130},...",2375776.5


Outlier rows


,display_info,name,displayName,owner_id,car_id,run_id,data_type_id,mods,run_data,total_power_normalized
35,{'username': 'JRink'},JRink HP 5 2,JRink HP (mtc stage 4),5,5,2,1,"{'2': 'race', '4': 'td05', '5': 'mtc stage 4'}","[{'x': 3300, 'y': 202}, {'x': 3400, 'y': 222},...",2482500.0


In [6]:
# for torque
torque_start, torque_end = summarize_trials_bounds(torque_trials['run_data'], 'torque')
print('Integration bounds (median q2): ({}, {})'.format(torque_start, torque_end))
torque_trials[integral_col_name] = torque_trials['run_data'].apply(lambda r: definite_discrete_integral(r, torque_start, torque_end))
torque_trials.head()

torque start RPM summary: {'min': 1916, 'q1': 2400.0, 'q2': 2536.0, 'q3': 2882.0, 'max': 3300, 'outliers': [1012]}
torque end RPM summary: {'min': 6399, 'q1': 6758.0, 'q2': 6961.0, 'q3': 7100.0, 'max': 7563, 'outliers': [7950, 6103]}
Integration bounds (median q2): (2536.0, 6961.0)


,display_info,name,displayName,owner_id,car_id,run_id,data_type_id,mods,run_data,total_power_normalized
1,{'username': '11secondFWD'},11secondFWD TQ 9 1,11secondFWD TQ (17g),9,9,1,2,"{'4': 'td04', '5': '17g'}","[{'x': 2514, 'y': 118}, {'x': 2619, 'y': 143},...",1290045.0
6,{'username': 'AdamVR4'},AdamVR4 TQ 8 1,AdamVR4 TQ (evo3 16g),8,8,1,2,"{'1': '30', '2': 'race', '4': 'td05', '5': 'ev...","[{'x': 3160, 'y': 232}, {'x': 3298, 'y': 252},...",1422245.5
8,{'username': 'BaadVR4'},BaadVR4 TQ 12 1,BaadVR4 TQ (evo3 16g),12,12,1,2,"{'1': '24', '2': 'pump', '4': 'td05', '5': 'ev...","[{'x': 2856, 'y': 226}, {'x': 3339, 'y': 283},...",1781925.5
11,{'username': 'boostaddict'},boostaddict TQ 2 1,boostaddict TQ (evo3 16g),2,2,1,2,"{'1': '36', '2': 'race', '3': '3.0', '4': 'td0...","[{'x': 2850, 'y': 176}, {'x': 2900, 'y': 180},...",1913475.0
16,{'username': 'chansigril'},chansigril TQ 48 1,chansigril TQ (13t),44,48,1,2,"{'4': 'td04', '5': '13t'}","[{'x': 2495, 'y': 159}, {'x': 2521, 'y': 296},...",1686407.0


In [7]:
# 5-stat summary on the torque integrals
torque_integral_summary = q_summary_dataframe(torque_trials, integral_col_name)
print_df_summary(torque_integral_summary)

Min row


,display_info,name,displayName,owner_id,car_id,run_id,data_type_id,mods,run_data,total_power_normalized
31,{'username': 'J-Groove'},J-Groove TQ 45 1,J-Groove TQ (9b),42,45,1,2,"{'1': '12', '2': 'pump', '3': '3.0', '4': 'td0...","[{'x': 2525, 'y': 216}, {'x': 2941, 'y': 256},...",924178.5


q1 row


,display_info,name,displayName,owner_id,car_id,run_id,data_type_id,mods,run_data,total_power_normalized
28,{'username': 'Intropy'},Intropy TQ 11 1,Intropy TQ (evo3 16g),11,11,1,2,"{'1': '18', '2': 'e85', '3': '3.0', '4': 'td05...","[{'x': 3112, 'y': 205}, {'x': 3328, 'y': 228},...",1487767.5


q2 row


,display_info,name,displayName,owner_id,car_id,run_id,data_type_id,mods,run_data,total_power_normalized
67,{'username': 'Ray Pampena'},Ray Pampena TQ 14 2,Ray Pampena TQ (dr650),14,14,2,2,"{'1': '25', '2': 'race', '3': '3.0', '4': 'td0...","[{'x': 2493, 'y': 211}, {'x': 2679, 'y': 233},...",1643524.5


q3 row


,display_info,name,displayName,owner_id,car_id,run_id,data_type_id,mods,run_data,total_power_normalized
34,{'username': 'JRink'},JRink TQ 5 1,JRink TQ (mtc stage 4),5,5,1,2,"{'2': 'pump', '4': 'td05', '5': 'mtc stage 4'}","[{'x': 3100, 'y': 291}, {'x': 3200, 'y': 306},...",2022550.0


Max row


,display_info,name,displayName,owner_id,car_id,run_id,data_type_id,mods,run_data,total_power_normalized
46,{'username': 'JRink'},JRink TQ 5 7,JRink TQ (fp 68hta),5,5,7,2,"{'2': 'race', '4': 'td05', '5': 'fp 68hta'}","[{'x': 2600, 'y': 173}, {'x': 2700, 'y': 212},...",2561250.0


## mods translation

In [8]:
mods

,mod_type_id,mod_name,mod_default,mod_type_type,mod_checked
0,1,Boost,0,range,NaN
1,2,Fuel,pump gas,fixed,NaN
2,3,Liter,3.0,fixed,NaN
3,4,Turbo Class,0,fixed,NaN
4,5,Turbo Name,0,fixed,NaN
5,6,Other,0,fixed,NaN
6,7,Injection,0,boolean,NaN
7,8,Nitrous,0,boolean,NaN
8,9,Correction,0,fixed,NaN
9,10,Heads,0,fixed,NaN


In [54]:
# mods spec is a dictionary e.g. only pump gas {'fuel':'gas'}
# filter out mods regards missing data as false - e.g. f('fuel':'gas', 'heads':'stock') -> wont include 'turbo_class: 3'

# value looks for match, None specifies not having that mod, and omission means keep any value,
# e.g. no 'displacement' in spec means any displacement is fine
td04_spec = {
    'turbo_class':'td04',# maybe make this the median boost: see above for stats on rows
}
stock_spec = {
    'injection':None,
    'nitrous':None,
    'heads':'stock',
    # counting not specified as stock
    'heads':None,
}
def filter_out_mods(trials, mods_spec):
    for idx, r in trials.iterrows():
        for k, v in mods_spec.items():
            specd_none = lambda k, v: k not in mods_spec and v is None
            specd_val = lambda k, v: k == v
            comp_func = (specd_none if k )
            ins = [(k == v) for k, v in mods_spec.items()]
            print(ins)
    return pd.DataFrame(filtered)

stock_engine = filter_out_mods(hp_trials, stock_spec)
#stock_engine['mods']

[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, Fa

NameError: name 'filtered' is not defined

## Question 1 - Best 1/8th, 1/2 mile
*Which run would be fastest in an 1/8th or 1/2 mile drag race?*

Drag races take place over 1/8th, 1/4, and 1/2 mile drag strips. 1/8 mile times are usually around 7 seconds - which means that the most important part of a drag race is right off the line. This means that we want the most "pull" off the line, i.e. the most torque.

The difference between 1/8th and 1/2 mile in this case is in what RPM range we care about - in the shorter 1/8th mile, you do not have the time to reach a higher RPM. So, for our project, we will define the ranges by statistial summary:

1/8th mile: [median start RPM, 4000]

1/2 mile: [median start RPM, 7000]

For each range, to find the best hypothetical drag car:
- **Calculate the total force for each torque run** by taking the integral on the specified bounds.
- Sort by the calculated total force, highest to lowest. **1st run will be the highest total force in the "off the line" RPM range**.

## Question 2 - "Bang for the Buck" fuel
*Which of the different types of fuel (pump gas, e85 ethanol fuel, 100 octane race fuel) provides the most power versus cost?*

In [10]:
mods.head()

,mod_type_id,mod_name,mod_default,mod_type_type,mod_checked
0,1,Boost,0,range,NaN
1,2,Fuel,pump gas,fixed,NaN
2,3,Liter,3.0,fixed,NaN
3,4,Turbo Class,0,fixed,NaN
4,5,Turbo Name,0,fixed,NaN


Total power over each run has been calculated. For a fair comparison, filter out horsepower runs with:
- stock head
- **same boost level** (either naturally aspirated/no turbo (0 boost) or most common, e.g. 20 psi - more boost means more power)
- **same turbo class** (any class will do, but choose the most common to have more data points)
- **similar displacement** higher displacement means more power. Filter by similar ranges, e.g. [3.0, 3.1] or [3.5, 3.7].

...then group by the fuel type. For each group, find the ratio of total power over fuel cost per gallon. The highest ratio will be the most powerful fuel per dollar. See the first deliverable for links to fuel prices.

## Question 3 - Top 10 Turbocharger Comparison
*Which of the 10 most popular turbochargers has the most lag?*

Turbochager lag is the "bump" in power delivery when boost kicks in. Looking at dyno charts, you can intuitively see the lag as those runs where.

The following function calculates the gradient for a run, i.e a list of the discrete derivatives between each point.

In [11]:
def discrete_gradient(run_data):
    run_x = [p['x'] for p in run_data]
    run_y = [p['y'] for p in run_data]
    return list(np.gradient(run_y, run_x))

In the example, the first element of the gradient is the difference between the first 2 points - so overall, if the run has *n* points, the gradient has *n-1* points. Remember - a negative number in the gradient describes a decrease in power.

The max element in the gradient is the maximum 'jump' - the max of the gradient for each comparable run will be the quantitative measure of turbo lag. 

In [12]:
# example of gradient for a single run
print_n_el = 5
run = hp_trials.iloc[0]['run_data']
print('Run data: {}...'.format(run[:print_n_el]))
print('Run gradient: {}...'.format(discrete_gradient(run)[:print_n_el]))
print('Max jump: {}'.format(max(discrete_gradient(run))))

Run data: [{'x': 2510, 'y': 53}, {'x': 2921, 'y': 107}, {'x': 2976, 'y': 109}, {'x': 3866, 'y': 214}, {'x': 4160, 'y': 257}]...
Run gradient: [0.13138686131386862, 0.0475788238577628, 0.04111365122601076, 0.13923603149124808, 0.16683480896161063]...
Max jump: 0.16683480896161063


So, to find the top 10 turbos by lag:
- **filter by same turbo class** - turbos are comparable by class - td03 turbo and td04 turbo is not a fair comparison.
- **filter by similar mods** (same heads, similar displacement)
- **find the max of the gradient for each run**
- **sort from least to highest max gradient value**, then take the top 10.

In [13]:
max_gradients = [max(discrete_gradient(r['run_data'])) for idx, r in hp_trials.iterrows()]
hp_trials['max_jump'] = max_gradients

hp_trials

,display_info,name,displayName,owner_id,car_id,run_id,data_type_id,mods,run_data,total_power_normalized,max_jump
0,{'username': '11secondFWD'},11secondFWD HP 9 1,11secondFWD HP (17g),9,9,1,1,"{'4': 'td04', '5': '17g'}","[{'x': 2510, 'y': 53}, {'x': 2921, 'y': 107}, ...",1023050.0,0.166835
2,{'username': '2root4u'},2root4u HP 18 1,2root4u HP (gt368),18,18,1,1,"{'1': '25', '4': 'td04', '5': 'gt368'}","[{'x': 3564, 'y': 210}, {'x': 3942, 'y': 262},...",1354755.0,0.217312
3,{'username': '3sx'},3sx HP 6 1,3sx HP (GT35),6,6,1,1,"{'4': 'td05', '5': 'GT35', '8': '100'}","[{'x': 4200, 'y': 200}, {'x': 4800, 'y': 370},...",1579750.0,0.356667
4,{'username': '97vr4'},97vr4 HP 15 1,97vr4 HP (19t),15,15,1,1,"{'1': '25', '4': 'td04', '5': '19t'}","[{'x': 3457, 'y': 201}, {'x': 3686, 'y': 244},...",1820158.0,0.451739
5,{'username': 'AdamVR4'},AdamVR4 HP 8 1,AdamVR4 HP (evo3 16g),8,8,1,1,"{'1': '30', '2': 'race', '4': 'td05', '5': 'ev...","[{'x': 3166, 'y': 144}, {'x': 3516, 'y': 189},...",1308968.5,0.303997
...,...,...,...,...,...,...,...,...,...,...,...
112,{'username': 'vr4usmc203'},vr4usmc203 HP 52 2,vr4usmc203 HP (13t),48,52,2,1,"{'2': 'race', '4': 'td04', '5': '13t', '11': '...","[{'x': 2758, 'y': 150}, {'x': 3150, 'y': 210},...",1366323.0,0.497911
114,{'username': 'widowmaker'},widowmaker HP 44 1,widowmaker HP (evo3 16g),41,44,1,1,"{'1': '19', '2': 'pump', '3': '3.5', '4': 'td0...","[{'x': 3677, 'y': 207}, {'x': 3923, 'y': 247},...",1234882.0,0.279197
115,{'username': 'Wraith'},Wraith HP 22 1,Wraith HP (dr650r),22,22,1,1,"{'1': '19', '4': 'td04', '5': 'dr650r'}","[{'x': 3824, 'y': 217}, {'x': 3891, 'y': 229},...",1058331.0,0.286557
116,{'username': 'wsrtt'},wsrtt HP 27 1,wsrtt HP (9b),27,27,1,1,"{'1': '21', '4': 'td04', '5': '9b', '7': '50/50'}","[{'x': 3023, 'y': 203}, {'x': 3165, 'y': 222},...",1041859.0,0.142057
